In [6]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

# Import API key
from doe_api import doe_key

In [64]:
#demographics

url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
params1 = {
    'api_key': doe_key,
    'per_page': 100,
    'fields': "latest.root.id,latest.root.location.lon,latest.root.location.lat,latest.school.name,"+
    
    "latest.school.zip,latest.student.size,latest.root.id,latest.school.region_id,"+
    "latest.aid.pell_grant_rate,latest.school.faculty_salary,latest.school.ft_faculty_rate,"+
    
    #admissions test requirements
    "latest.admissions.test_requirements,"+
    'latest.admissions.admission_rate.overall,'+
    
    #completion rate demographics
    'latest.completion.completion_rate_4yr_150nt,'+
    "latest.completion.completion_rate_4yr_150_nhpi,"+ 
    "latest.completion.completion_rate_4yr_150_aian,latest.completion.completion_rate_4yr_150_asian,"+
    "latest.completion.completion_rate_4yr_150_hispanic,latest.completion.completion_rate_4yr_150_black,"+
    "latest.completion.completion_rate_4yr_150_white,"+
    
    
    #student demographics
    "latest.student.demographics.race_ethnicity.white,latest.student.demographics.race_ethnicity.hispanic,"+
    "latest.student.demographics.race_ethnicity.black,latest.student.demographics.race_ethnicity.api,"+
    "latest.student.demographics.race_ethnicity.aian,latest.student.demographics.women,latest.student.demographics.men,"+
    
    #median debt per demographic
    "latest.aid.median_debt.income.30001_75000,latest.aid.median_debt.income.0_30000,latest.aid.median_debt.income.greater_than_75000,"+
    "latest.aid.median_debt.noncompleters,latest.aid.median_debt.completers.overall,latest.aid.median_debt.male_students,"+
    "latest.aid.median_debt.independent_students,latest.aid.median_debt.female_students,latest.aid.median_debt.dependent_students,"+
    
    #religious affiliation
    "latest.school.religious_affiliation,"+
    
    #undergrads age 25 and above
    "latest.student.share_25_older"+
    
    #page marker
    'page=0'
    
    
    
}

In [65]:
# Initialize an empty list to store the data
demographics_data = []
#starting page is 0
params1['page'] = 0

#loop that will loop through the page until we reach page 6
while params1['page'] < 7:
    #api call
    response = requests.get(url, params=params1).json()
    next_page_results = response.get("results", [])
    demographics_data.extend(next_page_results)
    #to go to the next page of results
    params1['page'] += 1

In [89]:
demographics_df = pd.DataFrame(demographics_data)
demographics_df = demographics_df.fillna(0)
demographics_df = demographics_df.drop(columns = ['latest.school.zip','latest.aid.pell_grant_rate',
                                                  'latest.school.faculty_salary',
                                                  'latest.school.ft_faculty_rate',
                                                 'latest.school.religious_affiliation',
                                                 'latest.student.demographics.race_ethnicity.api_2000',
                                                 'latest.school.region_id',
                                                 'latest.aid.median_debt.dependent_students',
                                                'latest.completion.completion_rate_4yr_150_white',
                                                 'latest.completion.completion_rate_4yr_150_nhpi',
                                                 'latest.completion.completion_rate_4yr_150_aian',
                                                 'latest.completion.completion_rate_4yr_150_asian',
                                                 'latest.completion.completion_rate_4yr_150_hispanic',
                                                 'latest.completion.completion_rate_4yr_150_black',
                                                 'latest.aid.median_debt.income.30001_75000',
                                                 'latest.aid.median_debt.income.0_30000',
                                                 'latest.aid.median_debt.income.greater_than_75000',
                                                  'latest.aid.median_debt.noncompleters',
                                                 'latest.aid.median_debt.completers.overall',
                                                  'latest.aid.median_debt.male_students',
                                                  'latest.aid.median_debt.independent_students',
                                                 'latest.aid.median_debt.female_students'])

In [90]:
#test required:
#1	Required
#2	Recommended
#3	Neither required nor recommended
#4	Do not know
#5	Considered but not required
#independent student == no financial support from parents or guardian 

demographics_clean = demographics_df.rename(columns={
    'latest.school.name':'School Name',
    'latest.student.size':'Student Count',
    'latest.admissions.test_requirements':'Standardized Test required? (1:Required, 2:Recommended, 3:Neither required nor recommended , 4:Do not know, 5: Considered but not required)',
    'latest.admissions.admission_rate.overall': 'Admission Rate',
    'latest.completion.completion_rate_4yr_150nt':'4Yr Uni Completion Rates',
    'latest.student.demographics.race_ethnicity.white': '% White Students',
    'latest.student.demographics.race_ethnicity.hispanic':'% Hispanic Students',
    'latest.student.demographics.race_ethnicity.black': '% Black Students',
    'latest.student.demographics.race_ethnicity.aian': '% AIAN Students',
    'latest.student.demographics.women':'% Female Students',
    'latest.student.demographics.men':'% Male Students'})

In [91]:
demographics_clean['Admission Rate'] = demographics_clean['Admission Rate'] * 100
demographics_clean['4Yr Uni Completion Rates'] = demographics_clean['4Yr Uni Completion Rates'] * 100
demographics_clean['% White Students'] = demographics_clean['% White Students'] * 100
demographics_clean['% Hispanic Students'] = demographics_clean['% Hispanic Students'] * 100
demographics_clean['% Black Students'] = demographics_clean['% Black Students'] * 100
demographics_clean['% AIAN Students'] = demographics_clean['% AIAN Students'] * 100
demographics_clean['% Female Students'] = demographics_clean['% Female Students'] * 100
demographics_clean['% Male Students'] = demographics_clean['% Male Students'] * 100

In [92]:
demographics_clean

,School Name,Student Count,"Standardized Test required? (1:Required, 2:Recommended, 3:Neither required nor recommended , 4:Do not know, 5: Considered but not required)",Admission Rate,4Yr Uni Completion Rates,% White Students,% Hispanic Students,% Black Students,% AIAN Students,% Female Students,% Male Students
0,Alabama A & M University,5098.0,5.0,71.60,28.07,1.84,1.14,89.78,0.27,59.40,40.60
1,University of Alabama at Birmingham,13284.0,5.0,88.54,62.45,52.97,6.69,24.58,0.21,62.10,37.90
2,Amridge University,251.0,0.0,0.00,44.44,24.70,4.38,69.32,0.00,68.53,31.47
3,University of Alabama in Huntsville,7358.0,1.0,73.67,60.72,71.96,6.10,8.71,0.80,40.47,59.53
4,Alabama State University,3495.0,5.0,97.99,28.43,1.52,1.29,92.59,0.17,64.95,35.05
...,...,...,...,...,...,...,...,...,...,...,...
695,Robert Morgan Educational Center and Technical...,435.0,0.0,0.00,0.00,10.11,60.00,26.90,0.00,47.59,52.41
696,Rollins College,2501.0,2.0,48.69,70.59,56.74,18.87,5.44,0.12,59.94,40.06
697,First Coast Technical College,379.0,0.0,0.00,0.00,68.07,9.76,17.15,0.00,53.83,46.17
698,Saint Leo University,6456.0,5.0,70.66,46.25,30.59,17.77,22.92,0.51,61.03,38.97


In [43]:
#By Income

url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
params2 = {
    'api_key': doe_key,
    'per_page': 250,
    'fields': "latest.root.id,latest.root.location.lon,latest.root.location.lat,latest.school.name,"+
    
    #median income
    "latest.student.demographics.median_family_income,latest.student.demographics.median_hh_income"
    
    #low income
    #general completion
    "latest.completion.title_iv.low_inc.completed_by.2yrs,latest.completion.title_iv.low_inc.completed_by.3yrs,"+
    "latest.completion.title_iv.low_inc.completed_by.4yrs,latest.completion.title_iv.low_inc.completed_by.6yrs,"+
    "latest.completion.title_iv.low_inc.completed_by.8yrs,"+
    #general withdrew
    "latest.completion.title_iv.low_inc.withdrawn_by.2yrs,latest.completion.title_iv.low_inc.withdrawn_by.3yrs,"+
    "latest.completion.title_iv.low_inc.withdrawn_by.4yrs,latest.completion.title_iv.low_inc.withdrawn_by.6yrs,"+
    "latest.completion.title_iv.low_inc.withdrawn_by.8yrs,"+
    #transferred and withdrew
    "latest.completion.title_iv.low_inc.transf_withdrawn_4yr_by.2yrs,latest.completion.title_iv.low_inc.transf_withdrawn_4yr_by.3yrs,"+
    "latest.completion.title_iv.low_inc.transf_withdrawn_4yr_by.4yrs,latest.completion.title_iv.low_inc.transf_withdrawn_4yr_by.6yrs,"+
    "latest.completion.title_iv.low_inc.transf_withdrawn_4yr_by.8yrs,"+
    #transferred and completed
    "latest.completion.title_iv.low_inc.transf_completed_4yr_by.2yrs,latest.completion.title_iv.low_inc.transf_completed_4yr_by.3yrs,"+
    "latest.completion.title_iv.low_inc.transf_completed_4yr_by.4yrs,latest.completion.title_iv.low_inc.transf_completed_4yr_by.6yrs,"+
    "latest.completion.title_iv.low_inc.transf_completed_4yr_by.8yrs,"+
    
    #middle income
    #general completion
    "latest.completion.title_iv.mid_inc.completed_by.2yrs,latest.completion.title_iv.mid_inc.completed_by.3yrs,"+
    "latest.completion.title_iv.mid_inc.completed_by.4yrs,latest.completion.title_iv.mid_inc.completed_by.6yrs,"+
    "latest.completion.title_iv.mid_inc.completed_by.8yrs,"+
    #general withdrew
    "latest.completion.title_iv.mid_inc.withdrawn_by.2yrs,latest.completion.title_iv.mid_inc.withdrawn_by.3yrs,"+
    "latest.completion.title_iv.mid_inc.withdrawn_by.4yrs,latest.completion.title_iv.mid_inc.withdrawn_by.6yrs,"+
    "latest.completion.title_iv.mid_inc.withdrawn_by.8yrs,"+
    #transferred and withdrew
    "latest.completion.title_iv.mid_inc.transf_withdrawn_4yr_by.2yrs,latest.completion.title_iv.mid_inc.transf_withdrawn_4yr_by.3yrs,"+
    "latest.completion.title_iv.mid_inc.transf_withdrawn_4yr_by.4yrs,latest.completion.title_iv.mid_inc.transf_withdrawn_4yr_by.6yrs,"+
    "latest.completion.title_iv.mid_inc.transf_withdrawn_4yr_by.8yrs,"+
    #transferred and completed
    "latest.completion.title_iv.mid_inc.transf_completed_4yr_by.2yrs,latest.completion.title_iv.mid_inc.transf_completed_4yr_by.3yrs,"+
    "latest.completion.title_iv.mid_inc.transf_completed_4yr_by.4yrs,latest.completion.title_iv.mid_inc.transf_completed_4yr_by.6yrs,"+
    "latest.completion.title_iv.mid_inc.transf_completed_4yr_by.8yrs,"+
    
    #high income
    #general completion
    "latest.completion.title_iv.high_inc.completed_by.2yrs,latest.completion.title_iv.high_inc.completed_by.3yrs,"+
    "latest.completion.title_iv.high_inc.completed_by.4yrs,latest.completion.title_iv.high_inc.completed_by.6yrs,"+
    "latest.completion.title_iv.high_inc.completed_by.8yrs,"+
    #general withdrew
    "latest.completion.title_iv.high_inc.withdrawn_by.2yrs,latest.completion.title_iv.high_inc.withdrawn_by.3yrs,"+
    "latest.completion.title_iv.high_inc.withdrawn_by.4yrs,latest.completion.title_iv.high_inc.withdrawn_by.6yrs,"+
    "latest.completion.title_iv.high_inc.withdrawn_by.8yrs,"+
    #transferred and withdrew
    "latest.completion.title_iv.high_inc.transf_withdrawn_4yr_by.2yrs,latest.completion.title_iv.high_inc.transf_withdrawn_4yr_by.3yrs,"+
    "latest.completion.title_iv.high_inc.transf_withdrawn_4yr_by.4yrs,latest.completion.title_iv.high_inc.transf_withdrawn_4yr_by.6yrs,"+
    "latest.completion.title_iv.high_inc.transf_withdrawn_4yr_by.8yrs,"+
    #transferred and completed
    "latest.completion.title_iv.high_inc.transf_completed_4yr_by.2yrs,latest.completion.title_iv.high_inc.transf_completed_4yr_by.3yrs,"+
    "latest.completion.title_iv.high_inc.transf_completed_4yr_by.4yrs,latest.completion.title_iv.high_inc.transf_completed_4yr_by.6yrs,"+
    "latest.completion.title_iv.high_inc.transf_completed_4yr_by.8yrs"+
    
    #page marker
    'page=0'
    
    
}

In [44]:
# Initialize an empty list to store the data
income_data = []
#starting page is 0
params2['page'] = 0

#loop that will loop through the page until we reach page 6
while params2['page'] < 7:
    #api call
    response = requests.get(url, params=params2).json()
    next_page_results = response.get("results", [])
    income_data.extend(next_page_results)
    #to go to the next page of results
    params2['page'] += 1

In [45]:
income_df = pd.DataFrame(income_data)
income_df

,latest.school.name,latest.student.demographics.median_family_income,latest.completion.title_iv.low_inc.completed_by.3yrs,latest.completion.title_iv.low_inc.completed_by.4yrs,latest.completion.title_iv.low_inc.completed_by.6yrs,latest.completion.title_iv.low_inc.completed_by.8yrs,latest.completion.title_iv.low_inc.withdrawn_by.2yrs,latest.completion.title_iv.low_inc.withdrawn_by.3yrs,latest.completion.title_iv.low_inc.withdrawn_by.4yrs,latest.completion.title_iv.low_inc.withdrawn_by.6yrs,...,latest.completion.title_iv.high_inc.withdrawn_by.8yrs,latest.completion.title_iv.high_inc.transf_withdrawn_4yr_by.2yrs,latest.completion.title_iv.high_inc.transf_withdrawn_4yr_by.3yrs,latest.completion.title_iv.high_inc.transf_withdrawn_4yr_by.4yrs,latest.completion.title_iv.high_inc.transf_withdrawn_4yr_by.6yrs,latest.completion.title_iv.high_inc.transf_withdrawn_4yr_by.8yrs,latest.completion.title_iv.high_inc.transf_completed_4yr_by.2yrs,latest.completion.title_iv.high_inc.transf_completed_4yr_by.3yrs,latest.completion.title_iv.high_inc.transf_completed_4yr_by.4yrs,latest.completion.title_iv.high_inc.transf_completed_4yr_by.6yrs
0,Alabama A & M University,23553.0,0.113772,0.197746,0.283042,0.265734,0.348643,0.319361,0.381148,0.327930,...,0.230000,NaN,NaN,NaN,0.126214,NaN,NaN,NaN,NaN,NaN
1,University of Alabama at Birmingham,34489.0,0.306597,0.440978,0.475979,0.477485,0.223259,0.247376,0.225126,0.208185,...,0.123103,NaN,NaN,NaN,0.021341,0.025295,NaN,0.031592,0.061856,0.114329
2,Amridge University,15033.0,NaN,NaN,NaN,0.104000,0.524823,0.532787,0.510638,0.508065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,University of Alabama in Huntsville,44787.0,0.293501,0.332075,0.435547,0.448582,0.255198,0.230608,0.283019,0.218750,...,0.138298,NaN,NaN,NaN,NaN,NaN,NaN,0.031599,0.064286,0.098039
4,Alabama State University,22080.0,0.120497,0.210593,0.297872,0.243494,0.352288,0.409938,0.345523,0.317447,...,0.168224,NaN,0.09322,NaN,NaN,0.112150,NaN,NaN,NaN,0.078431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Robert Morgan Educational Center and Technical...,16112.0,NaN,NaN,0.283105,0.275542,0.467153,0.598485,0.558442,0.456621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
696,Rollins College,43978.0,0.620915,0.590361,0.701087,0.652174,0.115152,0.111111,0.132530,0.103261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108844,NaN
697,First Coast Technical College,16017.0,0.591160,0.461538,0.548000,0.470588,NaN,0.292818,0.358974,0.248000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
698,Saint Leo University,22361.0,0.209421,0.224772,0.303931,0.346445,0.404938,0.342550,0.366183,0.269415,...,0.176781,NaN,NaN,0.042184,0.042035,0.026385,NaN,0.043732,0.084367,0.088496


In [68]:
#male/female
url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
params3 = {
    'api_key': doe_key,
    'per_page': 250,
    'fields': "latest.root.id,latest.root.location.lon,latest.root.location.lat,latest.school.name,"+
    
    #men/women only
    "latest.school.women_only,latest.school.men_only,"+
    
    #male 
    #general completion
    "latest.completion.title_iv.male.completed_by.2yrs,latest.completion.title_iv.male.completed_by.3yrs,"+
    "latest.completion.title_iv.male.completed_by.4yrs,latest.completion.title_iv.male.completed_by.6yrs,"+
    "latest.completion.title_iv.male.completed_by.8yrs,"+
    #general withdrew
    "latest.completion.title_iv.male.withdrawn_by.2yrs,latest.completion.title_iv.male.withdrawn_by.3yrs,"+
    "latest.completion.title_iv.male.withdrawn_by.4yrs,latest.completion.title_iv.male.withdrawn_by.6yrs,"+
    "latest.completion.title_iv.male.withdrawn_by.8yrs,"+
    #transferred and withdrew
    "latest.completion.title_iv.male.transf_withdrawn_4yr_by.2yrs,latest.completion.title_iv.male.transf_withdrawn_4yr_by.3yrs,"+
    "latest.completion.title_iv.male.transf_withdrawn_4yr_by.4yrs,latest.completion.title_iv.male.transf_withdrawn_4yr_by.6yrs,"+
    "latest.completion.title_iv.male.transf_withdrawn_4yr_by.8yrs,"+
    #transferred and completed
    "latest.completion.title_iv.male.transf_completed_4yr_by.2yrs,latest.completion.title_iv.male.transf_completed_4yr_by.3yrs,"+
    "latest.completion.title_iv.male.transf_completed_4yr_by.4yrs,latest.completion.title_iv.male.transf_completed_4yr_by.6yrs,"+
    "latest.completion.title_iv.male.transf_completed_4yr_by.8yrs,"+
    
    #female 
    #general completion
    "latest.completion.title_iv.female.completed_by.2yrs,latest.completion.title_iv.female.completed_by.3yrs,"+
    "latest.completion.title_iv.female.completed_by.4yrs,latest.completion.title_iv.female.completed_by.6yrs,"+
    "latest.completion.title_iv.female.completed_by.8yrs,"+
    #general withdrew
    "latest.completion.title_iv.female.withdrawn_by.2yrs,latest.completion.title_iv.female.withdrawn_by.3yrs,"+
    "latest.completion.title_iv.female.withdrawn_by.4yrs,latest.completion.title_iv.female.withdrawn_by.6yrs,"+
    "latest.completion.title_iv.female.withdrawn_by.8yrs,"+
    #transferred and withdrew
    "latest.completion.title_iv.female.transf_withdrawn_4yr_by.2yrs,latest.completion.title_iv.female.transf_withdrawn_4yr_by.3yrs,"+
    "latest.completion.title_iv.female.transf_withdrawn_4yr_by.4yrs,latest.completion.title_iv.female.transf_withdrawn_4yr_by.6yrs,"+
    "latest.completion.title_iv.female.transf_withdrawn_4yr_by.8yrs,"+
    #transferred and completed
    "latest.completion.title_iv.female.transf_completed_4yr_by.2yrs,latest.completion.title_iv.female.transf_completed_4yr_by.3yrs,"+
    "latest.completion.title_iv.female.transf_completed_4yr_by.4yrs,latest.completion.title_iv.female.transf_completed_4yr_by.6yrs,"+
    "latest.completion.title_iv.female.transf_completed_4yr_by.8yrs"+
    
    #page marker
    'page=0'
    
}

In [50]:
# Initialize an empty list to store the data
gender_completion = []
#starting page is 0
params3['page'] = 0

#loop that will loop through the page until we reach page 6
while params3['page'] < 7:
    #api call
    response = requests.get(url, params=params3).json()
    next_page_results = response.get("results", [])
    gender_completion.extend(next_page_results)
    #to go to the next page of results
    params3['page'] += 1

In [51]:
gender_completion_df = pd.DataFrame(gender_completion)
gender_completion_df

,latest.school.name,latest.school.women_only,latest.school.men_only,latest.completion.title_iv.male.completed_by.2yrs,latest.completion.title_iv.male.completed_by.3yrs,latest.completion.title_iv.male.completed_by.4yrs,latest.completion.title_iv.male.completed_by.6yrs,latest.completion.title_iv.male.completed_by.8yrs,latest.completion.title_iv.male.withdrawn_by.2yrs,latest.completion.title_iv.male.withdrawn_by.3yrs,...,latest.completion.title_iv.female.withdrawn_by.8yrs,latest.completion.title_iv.female.transf_withdrawn_4yr_by.2yrs,latest.completion.title_iv.female.transf_withdrawn_4yr_by.3yrs,latest.completion.title_iv.female.transf_withdrawn_4yr_by.4yrs,latest.completion.title_iv.female.transf_withdrawn_4yr_by.6yrs,latest.completion.title_iv.female.transf_withdrawn_4yr_by.8yrs,latest.completion.title_iv.female.transf_completed_4yr_by.2yrs,latest.completion.title_iv.female.transf_completed_4yr_by.3yrs,latest.completion.title_iv.female.transf_completed_4yr_by.4yrs,latest.completion.title_iv.female.transf_completed_4yr_by.6yrs
0,Alabama A & M University,0,0,0.040663,0.081267,0.176385,0.244556,0.255682,0.373494,0.353994,...,0.196691,0.043860,0.060104,0.066901,0.089764,0.119485,NaN,0.023834,0.030516,0.072441
1,University of Alabama at Birmingham,0,0,0.136760,0.288678,0.425299,0.509948,0.503590,0.223868,0.263613,...,0.123207,NaN,0.021436,0.026113,0.042112,0.035854,0.036267,0.042373,0.080119,0.091138
2,Amridge University,0,0,NaN,NaN,NaN,NaN,0.272727,0.600000,0.500000,...,0.283186,NaN,0.151515,NaN,0.137615,0.265487,NaN,NaN,NaN,NaN
3,University of Alabama in Huntsville,0,0,0.062350,0.235294,0.346726,0.481054,0.476274,0.219424,0.263427,...,0.101695,NaN,NaN,0.030547,0.045817,0.047458,NaN,0.056515,0.085209,0.127490
4,Alabama State University,0,0,0.054502,0.087866,0.194499,0.250000,0.223011,0.398104,0.464435,...,0.198385,0.050000,0.072050,0.063342,0.090317,0.155709,NaN,NaN,0.043127,0.074488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Robert Morgan Educational Center and Technical...,0,0,0.368421,NaN,NaN,0.182482,0.267327,0.484211,0.632075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
696,Rollins College,0,0,0.207792,0.588652,0.654762,0.642105,0.666667,0.097403,0.070922,...,0.099099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.076046
697,First Coast Technical College,0,0,0.736842,0.590909,NaN,0.541353,0.402062,NaN,0.295455,...,0.094241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
698,Saint Leo University,0,0,0.172117,0.268953,0.300317,0.397849,0.445068,0.387263,0.370939,...,0.197267,0.053055,0.050919,0.078489,0.093139,0.080273,NaN,0.035361,0.038424,0.053089


In [52]:
#dependent/independent
#first gen/non-first gen
url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
params4 = {
    'api_key': doe_key,
    'per_page': 250,
    'fields': "latest.root.id,latest.root.location.lon,latest.root.location.lat,latest.school.name,"+
    
    #students financially independent/dependent
    "latest.student.share_independent_lowincome.0_30000,latest.student.share_independent_students,latest.student.share_dependent_lowincome.0_300000,"+
    "latest.student.share_lowincome.0_30000,"+
    
    #first gen
    "latest.student.share_firstgeneration,latest.student.share_firstgeneration_parents.somecollege,"+
    "latest.student.share_firstgeneration_parents.middleschool,latest.student.share_firstgeneration_parents.highschool,"+
    
    #first gen tranfered to 4 year
    "latest.completion.title_iv.first_gen.transf_completed_4yr_by.2yrs,latest.completion.title_iv.first_gen.transf_completed_4yr_by.3yrs,"+
    "latest.completion.title_iv.first_gen.transf_completed_4yr_by.4yrs,latest.completion.title_iv.first_gen.transf_completed_4yr_by.6yrs,"+
    "latest.completion.title_iv.first_gen.transf_completed_4yr_by.8yrs,"+
    
    #via census data demographics (population of students by zipcode)
    "latest.student.demographics.share_bachelors_degree_age25.home_ZIP,latest.student.demographics.share_born_US.home_ZIP,"+
    "latest.student.demographics.share_white.home_ZIP,latest.student.demographics.share_hispanic.home_ZIP,"+
    "latest.student.demographics.share_black.home_ZIP,latest.student.demographics.share_asian.home_ZIP,"+
    "latest.student.demographics.share_professional_degree_age25.home_ZIP,"+
    
    #died
    "latest.completion.title_iv.not_first_gen.died_by.2yrs,latest.completion.title_iv.mid_inc.died_by.2yrs,"+
    "latest.completion.title_iv.low_inc.died_by.2yrs,latest.completion.title_iv.independ.died_by.2yrs,"+
    
    #independent
    #general completion
    "latest.completion.title_iv.independ.completed_by.2yrs,latest.completion.title_iv.independ.completed_by.3yrs,"+
    "latest.completion.title_iv.independ.completed_by.4yrs,latest.completion.title_iv.independ.completed_by.6yrs,"+
    "latest.completion.title_iv.independ.completed_by.8yrs,"+
    #general withdrew
    "latest.completion.title_iv.independ.withdrawn_by.2yrs,latest.completion.title_iv.independ.withdrawn_by.3yrs,"+
    "latest.completion.title_iv.independ.withdrawn_by.4yrs,latest.completion.title_iv.independ.withdrawn_by.6yrs,"+
    "latest.completion.title_iv.independ.withdrawn_by.8yrs,"+
    #transferred and withdrew
    "latest.completion.title_iv.independ.transf_withdrawn_4yr_by.2yrs,latest.completion.title_iv.independ.transf_withdrawn_4yr_by.3yrs,"+
    "latest.completion.title_iv.independ.transf_withdrawn_4yr_by.4yrs,latest.completion.title_iv.independ.transf_withdrawn_4yr_by.6yrs,"+
    "latest.completion.title_iv.independ.transf_withdrawn_4yr_by.8yrs,"+
    #transferred and completed
    "latest.completion.title_iv.independ.transf_completed_4yr_by.2yrs,latest.completion.title_iv.independ.transf_completed_4yr_by.3yrs,"+
    "latest.completion.title_iv.independ.transf_completed_4yr_by.4yrs,latest.completion.title_iv.independ.transf_completed_4yr_by.6yrs,"+
    "latest.completion.title_iv.independ.transf_completed_4yr_by.8yrs,"+
    
    #dependent
    #general completion
    "latest.completion.title_iv.depend.completed_by.2yrs,latest.completion.title_iv.depend.completed_by.3yrs,"+
    "latest.completion.title_iv.depend.completed_by.4yrs,latest.completion.title_iv.depend.completed_by.6yrs,"+
    "latest.completion.title_iv.depend.completed_by.8yrs,"+
    #general withdrew
    "latest.completion.title_iv.depend.withdrawn_by.2yrs,latest.completion.title_iv.depend.withdrawn_by.3yrs,"+
    "latest.completion.title_iv.depend.withdrawn_by.4yrs,latest.completion.title_iv.depend.withdrawn_by.6yrs,"+
    "latest.completion.title_iv.depend.withdrawn_by.8yrs,"+
    #transferred and withdrew
    "latest.completion.title_iv.depend.transf_withdrawn_4yr_by.2yrs,latest.completion.title_iv.depend.transf_withdrawn_4yr_by.3yrs,"+
    "latest.completion.title_iv.depend.transf_withdrawn_4yr_by.4yrs,latest.completion.title_iv.depend.transf_withdrawn_4yr_by.6yrs,"+
    "latest.completion.title_iv.depend.transf_withdrawn_4yr_by.8yrs,"+
    #transferred and completed
    "latest.completion.title_iv.depend.transf_completed_4yr_by.2yrs,latest.completion.title_iv.depend.transf_completed_4yr_by.3yrs,"+
    "latest.completion.title_iv.depend.transf_completed_4yr_by.4yrs,latest.completion.title_iv.depend.transf_completed_4yr_by.6yrs,"+
    "latest.completion.title_iv.depend.transf_completed_4yr_by.8yrs,"+
    
    #non-first gen
    #general completion
    "latest.completion.title_iv.not_first_gen.completed_by.2yrs,latest.completion.title_iv.not_first_gen.completed_by.3yrs,"+
    "latest.completion.title_iv.not_first_gen.completed_by.4yrs,latest.completion.title_iv.not_first_gen.completed_by.6yrs,"+
    "latest.completion.title_iv.not_first_gen.completed_by.8yrs,"+
    #general withdrew
    "latest.completion.title_iv.not_first_gen.withdrawn_by.2yrs,latest.completion.title_iv.not_first_gen.withdrawn_by.3yrs,"+
    "latest.completion.title_iv.not_first_gen.withdrawn_by.4yrs,latest.completion.title_iv.not_first_gen.withdrawn_by.6yrs,"+
    "latest.completion.title_iv.not_first_gen.withdrawn_by.8yrs,"+
    #transferred and withdrew
    "latest.completion.title_iv.not_first_gen.transf_withdrawn_4yr_by.2yrs,latest.completion.title_iv.not_first_gen.transf_withdrawn_4yr_by.3yrs,"+
    "latest.completion.title_iv.not_first_gen.transf_withdrawn_4yr_by.4yrs,latest.completion.title_iv.not_first_gen.transf_withdrawn_4yr_by.6yrs,"+
    "latest.completion.title_iv.not_first_gen.transf_withdrawn_4yr_by.8yrs,"+
    #transferred and completed
    "latest.completion.title_iv.not_first_gen.transf_completed_4yr_by.2yrs,latest.completion.title_iv.not_first_gen.transf_completed_4yr_by.3yrs,"+
    "latest.completion.title_iv.not_first_gen.transf_completed_4yr_by.4yrs,latest.completion.title_iv.not_first_gen.transf_completed_4yr_by.6yrs,"+
    "latest.completion.title_iv.not_first_gen.transf_completed_4yr_by.8yrs,"+
    
    #first gen
    #general completion
    "latest.completion.title_iv.first_gen.completed_by.2yrs,latest.completion.title_iv.first_gen.completed_by.3yrs,"+
    "latest.completion.title_iv.first_gen.completed_by.4yrs,latest.completion.title_iv.first_gen.completed_by.6yrs,"+
    "latest.completion.title_iv.first_gen.completed_by.8yrs,"+
    #general withdrew
    "latest.completion.title_iv.first_gen.withdrawn_by.2yrs,latest.completion.title_iv.first_gen.withdrawn_by.3yrs,"+
    "latest.completion.title_iv.first_gen.withdrawn_by.4yrs,latest.completion.title_iv.first_gen.withdrawn_by.6yrs,"+
    "latest.completion.title_iv.first_gen.withdrawn_by.8yrs,"+
    #transferred and withdrew
    "latest.completion.title_iv.first_gen.transf_withdrawn_4yr_by.2yrs,latest.completion.title_iv.first_gen.transf_withdrawn_4yr_by.3yrs,"+
    "latest.completion.title_iv.first_gen.transf_withdrawn_4yr_by.4yrs,latest.completion.title_iv.first_gen.transf_withdrawn_4yr_by.6yrs,"+
    "latest.completion.title_iv.first_gen.transf_withdrawn_4yr_by.8yrs,"+
    #transferred and completed
    "latest.completion.title_iv.first_gen.transf_completed_4yr_by.2yrs,latest.completion.title_iv.first_gen.transf_completed_4yr_by.3yrs,"+
    "latest.completion.title_iv.first_gen.transf_completed_4yr_by.4yrs,latest.completion.title_iv.first_gen.transf_completed_4yr_by.6yrs,"+
    "latest.completion.title_iv.first_gen.transf_completed_4yr_by.8yrs"+
    
    #page marker
    'page=0'
    
}

In [54]:
# Initialize an empty list to store the data
generational_completion = []
#starting page is 0
params4['page'] = 0

#loop that will loop through the page until we reach page 6
while params4['page'] < 7:
    #api call
    response = requests.get(url, params=params4).json()
    next_page_results = response.get("results", [])
    generational_completion.extend(next_page_results)
    #to go to the next page of results
    params4['page'] += 1

In [56]:
generational_completion_df = pd.DataFrame(generational_completion)
generational_completion_df

,latest.school.name,latest.student.share_independent_lowincome.0_30000,latest.student.share_independent_students,latest.student.share_dependent_lowincome.0_300000,latest.student.share_lowincome.0_30000,latest.student.share_firstgeneration,latest.student.share_firstgeneration_parents.somecollege,latest.student.share_firstgeneration_parents.middleschool,latest.student.share_firstgeneration_parents.highschool,latest.completion.title_iv.first_gen.transf_completed_4yr_by.2yrs,...,latest.completion.title_iv.first_gen.withdrawn_by.2yrs,latest.completion.title_iv.first_gen.withdrawn_by.3yrs,latest.completion.title_iv.first_gen.withdrawn_by.4yrs,latest.completion.title_iv.first_gen.withdrawn_by.6yrs,latest.completion.title_iv.first_gen.withdrawn_by.8yrs,latest.completion.title_iv.first_gen.transf_withdrawn_4yr_by.2yrs,latest.completion.title_iv.first_gen.transf_withdrawn_4yr_by.3yrs,latest.completion.title_iv.first_gen.transf_withdrawn_4yr_by.4yrs,latest.completion.title_iv.first_gen.transf_withdrawn_4yr_by.6yrs,latest.completion.title_iv.first_gen.transf_withdrawn_4yr_by.8yrs
0,Alabama A & M University,0.901587,0.098870,0.569140,0.602009,0.365828,0.634172,0.016422,0.349406,NaN,...,0.358268,0.313928,0.350794,0.331707,0.271577,0.044882,0.058954,0.073016,0.087805,0.107020
1,University of Alabama at Birmingham,0.594203,0.321409,0.348710,0.427613,0.341224,0.658776,0.022267,0.318957,0.032513,...,0.214851,0.239803,0.218299,0.186275,0.166758,NaN,0.025997,0.026752,0.039760,0.046230
2,Amridge University,NaN,0.895270,NaN,0.773649,0.512500,0.487500,0.050000,0.462500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,University of Alabama in Huntsville,0.688852,0.251570,0.269016,0.374634,0.310132,0.689868,0.019824,0.290308,NaN,...,0.238883,0.237214,0.279236,0.193421,0.159148,0.021717,NaN,0.033413,0.038158,0.045113
4,Alabama State University,0.921933,0.107428,0.577629,0.614617,0.343434,0.656566,0.018825,0.324610,NaN,...,0.343182,0.388190,0.331683,0.309307,0.274924,0.043182,0.060987,0.074257,0.093772,0.151813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Robert Morgan Educational Center and Technical...,0.889535,0.463612,0.633166,0.752022,0.515571,0.484429,0.051903,0.463668,NaN,...,0.448052,0.595890,0.578652,NaN,NaN,NaN,NaN,NaN,NaN,NaN
696,Rollins College,0.647799,0.198750,0.287051,0.358750,0.237533,0.762467,NaN,NaN,NaN,...,0.108434,NaN,0.124464,0.131474,0.105263,NaN,NaN,NaN,NaN,NaN
697,First Coast Technical College,0.848797,0.676744,0.647482,0.783721,0.505464,0.494536,0.051913,0.453552,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
698,Saint Leo University,0.673750,0.788684,0.435301,0.623362,0.491043,0.508957,0.043504,0.447539,NaN,...,0.366558,0.329728,0.338883,0.259388,0.209581,0.052004,0.055949,0.079918,0.092490,0.082502


In [57]:
#Academics
url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
params5 = {
    'api_key': doe_key,
    'per_page': 500,
    'fields': "latest.root.id,latest.root.location.lon,latest.root.location.lat,latest.school.name,"+
    
    "latest.aid.federal_loan_rate,latest.cost.title_iv.public.all,latest.cost.title_iv.private.all,"+
    "latest.academics.program_reporter.programs_offered,latest.school.tuition_revenue_per_fte,"+
    
    #enrollment
    "latest.student.enrollment.all,"+
    
    #weird school specifics
    "latest.school.state_fips,latest.root.fed_sch_cd"+
    
    #tuition fees
    "latest.cost.tuition.out_of_state,latest.cost.tuition.in_state,"+
    
    #highest degree awarded
    "latest.school.degrees_awarded.highest,"+
    
    #test scores
    #sat
    "latest.admissions.sat_scores.midpoint.writing,latest.admissions.sat_scores.midpoint.math,"+
    "latest.admissions.sat_scores.midpoint.critical_reading,"+
    "latest.admissions.sat_scores.average.by_ope_id,latest.admissions.sat_scores.average.overall,"+
    #act
    "latest.admissions.act_scores.midpoint.writing,latest.admissions.act_scores.midpoint.math,"+
    "latest.admissions.act_scores.midpoint.english,latest.admissions.act_scores.midpoint.cumulative"
    
    #retention rate
    "latest.student.retention_rate.lt_four_year.part_time_pooled,latest.student.retention_rate.four_year.part_time_pooled,"+
    "latest.student.retention_rate.lt_four_year.full_time_pooled,latest.student.retention_rate.four_year.full_time_pooled,"+
    
    #cost
    "latest.cost.otherexpense.withfamily,latest.cost.roomboard.oncampus,latest.cost.otherexpense.oncampus,"+
    "latest.cost.roomboard.offcampus,latest.cost.otherexpense.offcampus,latest.cost.booksupply,"+
    
    #control of institution
    "latest.school.ownership"
    
    #less than 4 year first-time full-time
    "latest.completion.completion_rate_less_than_4yr_150nt_pooled,latest.completion.completion_rate_less_than_4yr_150nt,"+
    "latest.completion.completion_rate_l4yr_150_white,latest.completion.completion_rate_l4yr_150_2ormore,"+
    "latest.completion.completion_rate_l4yr_150_nhpi,latest.completion.completion_rate_l4yr_150_hispanic,"+
    "latest.completion.completion_rate_l4yr_150_black,latest.completion.completion_rate_l4yr_150_asian,"+
    "latest.completion.completion_rate_l4yr_150_aian,"+
    
    #4 year first-time full-time
    "latest.completion.completion_rate_4yr_150_white,latest.completion.completion_rate_4yr_150_2ormore,"+
    "latest.completion.completion_rate_4yr_150_nhpi,latest.completion.completion_rate_4yr_150_hispanic,"+
    "latest.completion.completion_rate_4yr_150_black,latest.completion.completion_rate_4yr_150_asian,"+
    "latest.completion.completion_rate_4yr_150_aian,latest.completion.completion_rate_4yr_150nt_pooled,"+
    "latest.completion.completion_rate_4yr_150nt"+
    
    #page marker
    'page=0'
    
}

In [59]:
# Initialize an empty list to store the data
academics = []
#starting page is 0
params5['page'] = 0

#loop that will loop through the page until we reach page 6
while params5['page'] < 7:
    #api call
    response = requests.get(url, params=params5).json()
    next_page_results = response.get("results", [])
    academics.extend(next_page_results)
    #to go to the next page of results
    params5['page'] += 1

In [60]:
academics_df = pd.DataFrame(academics)
academics_df

,latest.school.name,latest.aid.federal_loan_rate,latest.cost.title_iv.public.all,latest.cost.title_iv.private.all,latest.academics.program_reporter.programs_offered,latest.school.tuition_revenue_per_fte,latest.student.enrollment.all,latest.school.state_fips,latest.cost.tuition.in_state,latest.school.degrees_awarded.highest,...,latest.completion.completion_rate_l4yr_150_asian,latest.completion.completion_rate_l4yr_150_aian,latest.completion.completion_rate_4yr_150_white,latest.completion.completion_rate_4yr_150_2ormore,latest.completion.completion_rate_4yr_150_nhpi,latest.completion.completion_rate_4yr_150_hispanic,latest.completion.completion_rate_4yr_150_black,latest.completion.completion_rate_4yr_150_asian,latest.completion.completion_rate_4yr_150_aian,latest.completion.completion_rate_4yr_150nt_pooled
0,Alabama A & M University,0.6552,657.0,NaN,NaN,9243.0,None,1,10024.0,4,...,NaN,NaN,0.3043,0.3214,0.0000,0.2222,0.2805,0.0000,0.0000,0.2833
1,University of Alabama at Birmingham,0.4401,1053.0,NaN,NaN,11678.0,None,1,8568.0,4,...,NaN,NaN,0.6276,0.5000,NaN,0.5769,0.5565,0.8354,0.6000,0.6181
2,Amridge University,0.8423,NaN,NaN,NaN,14093.0,None,1,NaN,4,...,NaN,NaN,1.0000,NaN,NaN,NaN,0.2857,NaN,NaN,0.3846
3,University of Alabama in Huntsville,0.3578,515.0,NaN,NaN,8246.0,None,1,11488.0,4,...,NaN,NaN,0.6098,0.6000,NaN,0.6444,0.4845,0.8125,0.6667,0.5925
4,Alabama State University,0.7637,418.0,NaN,NaN,8535.0,None,1,11068.0,4,...,NaN,NaN,0.3636,0.0625,NaN,0.4286,0.2844,0.3333,0.0000,0.3015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Robert Morgan Educational Center and Technical...,0.0000,77.0,NaN,20.0,1479.0,None,12,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
696,Rollins College,0.3022,NaN,242.0,NaN,20478.0,None,12,54740.0,4,...,NaN,NaN,0.7210,0.6000,NaN,0.6667,0.8571,0.6000,0.5000,0.7590
697,First Coast Technical College,0.0000,19.0,NaN,17.0,4315.0,None,12,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
698,Saint Leo University,0.4543,NaN,527.0,NaN,11729.0,None,12,26050.0,4,...,NaN,NaN,0.4637,0.4211,NaN,0.4397,0.4301,0.7500,NaN,0.4830


In [67]:
url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
params6 = {
    'api_key': doe_key,
    'per_page': 250,
    'fields': "latest.root.id,latest.root.location.lon,latest.root.location.lat,latest.school.name,"+
    
    #school city / classifications
    "latest.school.city,latest.school.carnegie_undergrad,latest.school.carnegie_size_setting,"+
    "latest.school.carnegie_basic"
    
    #costs
    "latest.cost.attendance.academic_year,latest.cost.attendance.program_year"+
    
    #fafsa
    "latest.student.fafsa_sent.overall,"+
    
    #completion time by program
    "latest.academics.program_reporter.program_1.cip_6_digit.avg_month_completion,"+
    "latest.academics.program_reporter.program_2.cip_6_digit.avg_month_completion,"+
    "latest.academics.program_reporter.program_3.cip_6_digit.avg_month_completion,"+
    "latest.academics.program_reporter.program_4.cip_6_digit.avg_month_completion,"+
    "latest.academics.program_reporter.program_5.cip_6_digit.avg_month_completion,"+
    "latest.academics.program_reporter.program_6.cip_6_digit.avg_month_completion,"+
    
    #average net price
    "latest.cost.avg_net_price.public,latest.cost.avg_net_price.private,latest.cost.avg_net_price.other_academic_year,"+
    "latest.cost.avg_net_price.program_year,latest.cost.net_price.public.by_income_level.75001-110000,"+
    "latest.cost.net_price.program_reporter.by_income_level.75001-110000,latest.cost.net_price.private.by_income_level.75001-110000,"+
    "latest.cost.net_price.other_acad_calendar.by_income_level.75001-110000,latest.cost.net_price.public.by_income_level.75000-plus,"+
    "latest.cost.net_price.program_reporter.by_income_level.75000-plus,latest.cost.net_price.private.by_income_level.75000-plus,"+
    "latest.cost.net_price.other_acad_calendar.by_income_level.75000-plus,latest.cost.net_price.public.by_income_level.48001-75000,"+
    "latest.cost.net_price.program_reporter.by_income_level.48001-75000,latest.cost.net_price.private.by_income_level.48001-75000,"+
    "latest.cost.net_price.other_acad_calendar.by_income_level.48001-75000,latest.cost.net_price.public.by_income_level.30001-75000,"+
    "latest.cost.net_price.program_reporter.by_income_level.30001-75000,latest.cost.net_price.private.by_income_level.30001-75000,"+
    "latest.cost.net_price.other_acad_calendar.by_income_level.30001-75000,latest.cost.net_price.public.by_income_level.30001-48000,"+
    "latest.cost.net_price.program_reporter.by_income_level.30001-48000,latest.cost.net_price.private.by_income_level.30001-48000,"+
    "latest.cost.net_price.other_acad_calendar.by_income_level.30001-48000,latest.cost.net_price.public.by_income_level.110001-plus,"+
    "latest.cost.net_price.program_reporter.by_income_level.110001-plus,latest.cost.net_price.private.by_income_level.110001-plus,"+
    "latest.cost.net_price.other_acad_calendar.by_income_level.110001-plus,latest.cost.net_price.public.by_income_level.0-48000,"+
    "latest.cost.net_price.program_reporter.by_income_level.0-48000,latest.cost.net_price.private.by_income_level.0-48000,"+
    "latest.cost.net_price.other_acad_calendar.by_income_level.0-48000,latest.cost.net_price.public.by_income_level.0-30000,"+
    "latest.cost.net_price.program_reporter.by_income_level.0-30000,latest.cost.net_price.private.by_income_level.0-30000,"+
    "latest.cost.net_price.other_acad_calendar.by_income_level.0-30000,"+
    
    #admission rate
    "latest.admissions.admission_rate.overall"+
    
    #page marker
    'page=0'
}

In [65]:
# Initialize an empty list to store the data
cost_income_admission = []
#starting page is 0
params6['page'] = 0

#loop that will loop through the page until we reach page 6
while params6['page'] < 7:
    #api call
    response = requests.get(url, params=params6).json()
    next_page_results = response.get("results", [])
    cost_income_admission.extend(next_page_results)
    #to go to the next page of results
    params6['page'] += 1

In [66]:
cost_income_admis_df = pd.DataFrame(cost_income_admission)
cost_income_admis_df

,latest.school.name,latest.school.city,latest.school.carnegie_undergrad,latest.school.carnegie_size_setting,latest.academics.program_reporter.program_1.cip_6_digit.avg_month_completion,latest.academics.program_reporter.program_2.cip_6_digit.avg_month_completion,latest.academics.program_reporter.program_3.cip_6_digit.avg_month_completion,latest.academics.program_reporter.program_4.cip_6_digit.avg_month_completion,latest.academics.program_reporter.program_5.cip_6_digit.avg_month_completion,latest.academics.program_reporter.program_6.cip_6_digit.avg_month_completion,...,latest.cost.net_price.private.by_income_level.110001-plus,latest.cost.net_price.other_acad_calendar.by_income_level.110001-plus,latest.cost.net_price.public.by_income_level.0-48000,latest.cost.net_price.program_reporter.by_income_level.0-48000,latest.cost.net_price.private.by_income_level.0-48000,latest.cost.net_price.other_acad_calendar.by_income_level.0-48000,latest.cost.net_price.public.by_income_level.0-30000,latest.cost.net_price.program_reporter.by_income_level.0-30000,latest.cost.net_price.private.by_income_level.0-30000,latest.cost.net_price.other_acad_calendar.by_income_level.0-30000
0,Alabama A & M University,Normal,10,14,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,12210.0,None,NaN,None,12177.0,None,NaN,None
1,University of Alabama at Birmingham,Birmingham,9,15,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,14431.0,None,NaN,None,13862.0,None,NaN,None
2,Amridge University,Montgomery,5,6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
3,University of Alabama in Huntsville,Huntsville,15,13,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,14651.0,None,NaN,None,14654.0,None,NaN,None
4,Alabama State University,Montgomery,10,14,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,13180.0,None,NaN,None,12877.0,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Robert Morgan Educational Center and Technical...,Miami,-2,-2,18.0,24.0,16.0,12.0,24.0,12.0,...,NaN,None,12420.0,None,NaN,None,12297.0,None,NaN,None
696,Rollins College,Winter Park,12,11,NaN,NaN,NaN,NaN,NaN,NaN,...,36703.0,None,NaN,None,14884.0,None,NaN,None,14855.0,None
697,First Coast Technical College,Saint Augustine,-2,-2,14.0,12.0,7.0,12.0,12.0,6.0,...,NaN,None,19780.0,None,NaN,None,19715.0,None,NaN,None
698,Saint Leo University,Saint Leo,9,12,NaN,NaN,NaN,NaN,NaN,NaN,...,22440.0,None,NaN,None,15291.0,None,NaN,None,14671.0,None
